<a href="https://colab.research.google.com/github/muoyo/chicago-ridesharing/blob/master/notebooks/rideshare_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sodapy

In [3]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# !pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata('data.cityofchicago.org',
                 'Tk6RhuGAFvF9P4ehsysybj3IW',
                 username="mokome@gmail.com",
                 password="Ch1cago!!")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
client.timeout = 10000
results = client.get("m6dm-c72p", limit=1000000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trip_total,shared_trip_authorized,trips_pooled,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
0,033eb553a267c9672a16307749ba585b2279d950,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,320,1.386507,17031081500,17031080100,8,8,5,0,2.55,7.55,False,1,41.8925077809,-87.6262149064,"{'type': 'Point', 'coordinates': [-87.62621490...",41.9075200747,-87.6266589003,"{'type': 'Point', 'coordinates': [-87.62665890..."
1,06d0dc55591d6fafd44943dceb79db36f9e5f10b,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,319,1.403693,NaN,NaN,58,63,5,0,2.55,7.55,False,1,41.8173662079,-87.6988607973,"{'type': 'Point', 'coordinates': [-87.69886079...",41.795430631,-87.6964352325,"{'type': 'Point', 'coordinates': [-87.69643523..."
2,30d8197b8eee1730b0af2719dcdb13a64726bd6b,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,335,1.585585,NaN,NaN,44,44,5,0,2.55,7.55,False,1,41.7402057565,-87.6159695226,"{'type': 'Point', 'coordinates': [-87.61596952...",41.7402057565,-87.6159695226,"{'type': 'Point', 'coordinates': [-87.61596952..."
3,0b4c86af9dfac5bf734ae0a651e8e997bcc00264,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,778,5.879604,NaN,NaN,7,5,10,0,2.55,12.55,False,1,41.9226862843,-87.6494887289,"{'type': 'Point', 'coordinates': [-87.64948872...",41.9477915865,-87.6838349425,"{'type': 'Point', 'coordinates': [-87.68383494..."
4,322542beb412800793bd4eb7939202f8606246e1,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,342,1.054647,17031081500,17031839100,8,32,5,0,2.55,7.55,False,1,41.8925077809,-87.6262149064,"{'type': 'Point', 'coordinates': [-87.62621490...",41.8809944707,-87.6327464887,"{'type': 'Point', 'coordinates': [-87.63274648..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,042fdebf257a8746b6cbecba054e91a51c94c4a9,2019-09-27T19:45:00.000,2019-09-27T20:00:00.000,865,3.59310224672,17031030900,17031062100,77,6,7.5,0,2.55,10.05,False,1,41.9797955506,-87.6714455473,"{'type': 'Point', 'coordinates': [-87.67144554...",41.9426918444,-87.6517705068,"{'type': 'Point', 'coordinates': [-87.65177050..."
999996,60ff322051f0eed338ae1f6c5950f7f050f673d3,2019-09-27T19:45:00.000,2019-09-27T19:45:00.000,644,1.143872,17031080202,17031081401,8,8,5,0,2.55,7.55,True,3,41.9058577688,-87.6308650266,"{'type': 'Point', 'coordinates': [-87.63086502...",41.8950334495,-87.6197106717,"{'type': 'Point', 'coordinates': [-87.61971067..."
999997,00ffd26fdede03e27ce93c69adf5a0849c9be74b,2019-09-27T19:45:00.000,2019-09-27T19:45:00.000,576,2.19550851568,17031841100,17031839800,34,60,10,0,2.55,12.55,False,1,41.8510178239,-87.6350918563,"{'type': 'Point', 'coordinates': [-87.63509185...",41.8350679168,-87.6431623703,"{'type': 'Point', 'coordinates': [-87.64316237..."
999998,62dfc340528050d5aeae99eaf16e51ce2cced419,2019-09-27T19:45:00.000,2019-09-27T20:00:00.000,337,1.478123,NaN,NaN,4,5,5,0,2.55,7.55,False,1,41.9751709433,-87.6875155152,"{'type': 'Point', 'coordinates': [-87.68751551...",41.9477915865,-87.6838349425,"{'type': 'Point', 'coordinates': [-87.68383494..."


In [5]:
results_df.head()

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trip_total,shared_trip_authorized,trips_pooled,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
0,033eb553a267c9672a16307749ba585b2279d950,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,320,1.386507,17031081500,17031080100,8,8,5,0,2.55,7.55,False,1,41.8925077809,-87.6262149064,"{'type': 'Point', 'coordinates': [-87.62621490...",41.9075200747,-87.6266589003,"{'type': 'Point', 'coordinates': [-87.62665890..."
1,05f8c7b6ab9add701206b29fada5f6db1a358da3,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,515,2.716552,NaN,17031010702,NaN,1,7.5,0,2.55,10.05,False,1,NaN,NaN,NaN,42.0090182269,-87.6727239593,"{'type': 'Point', 'coordinates': [-87.67272395..."
2,120b0c6c8bc222c9cae064083a1e0152481a14d6,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,329,1.370964,NaN,NaN,39,38,5,0,2.55,7.55,True,1,41.8089162826,-87.5961833442,"{'type': 'Point', 'coordinates': [-87.59618334...",41.8129489392,-87.6178596758,"{'type': 'Point', 'coordinates': [-87.61785967..."
3,1413969321456405849e5f69b90f4efd543e02ed,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,397,1.439182,17031063000,17031060800,6,6,5,0,2.55,7.55,False,1,41.9363101308,-87.6515625922,"{'type': 'Point', 'coordinates': [-87.65156259...",41.9534000435,-87.6460070664,"{'type': 'Point', 'coordinates': [-87.64600706..."
4,1d0c3724f384cd00a95b28132d4ac3f7eba9506f,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,175,0.771651,NaN,NaN,24,24,2.5,0,2.55,5.05,False,1,41.9012069941,-87.6763559892,"{'type': 'Point', 'coordinates': [-87.67635598...",41.9012069941,-87.6763559892,"{'type': 'Point', 'coordinates': [-87.67635598..."


In [12]:
results_df.columns

Index(['trip_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds',
       'trip_miles', 'pickup_census_tract', 'dropoff_census_tract',
       'pickup_community_area', 'dropoff_community_area', 'fare', 'tip',
       'additional_charges', 'trip_total', 'shared_trip_authorized',
       'trips_pooled', 'pickup_centroid_latitude', 'pickup_centroid_longitude',
       'pickup_centroid_location', 'dropoff_centroid_latitude',
       'dropoff_centroid_longitude', 'dropoff_centroid_location'],
      dtype='object')

In [0]:
columns_to_use = ['trip_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds',
       'trip_miles', 'pickup_community_area', 'dropoff_community_area', 'fare', 'tip',
       'additional_charges', 'trip_total' ]

columns_to_drop = [ col for col in results_df.columns if col not in columns_to_use ]

In [15]:
df = results_df.drop(columns=columns_to_drop)
df.head()

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trip_total
0,033eb553a267c9672a16307749ba585b2279d950,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,320,1.386507,8,8,5,0,2.55,7.55
1,06d0dc55591d6fafd44943dceb79db36f9e5f10b,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,319,1.403693,58,63,5,0,2.55,7.55
2,30d8197b8eee1730b0af2719dcdb13a64726bd6b,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,335,1.585585,44,44,5,0,2.55,7.55
3,0b4c86af9dfac5bf734ae0a651e8e997bcc00264,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,778,5.879604,7,5,10,0,2.55,12.55
4,322542beb412800793bd4eb7939202f8606246e1,2019-09-30T23:45:00.000,2019-09-30T23:45:00.000,342,1.054647,8,32,5,0,2.55,7.55


In [0]:
df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])
df['trip_seconds'] = df['trip_seconds'].astype('int64')

for col in ['trip_miles', 'fare', 'tip', 'additional_charges', 'trip_total']:
    df[col] = df[col].astype(float)  

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 11 columns):
trip_id                   1000000 non-null object
trip_start_timestamp      1000000 non-null datetime64[ns]
trip_end_timestamp        1000000 non-null datetime64[ns]
trip_seconds              1000000 non-null int64
trip_miles                999994 non-null float64
pickup_community_area     935379 non-null object
dropoff_community_area    925497 non-null object
fare                      1000000 non-null float64
tip                       1000000 non-null float64
additional_charges        1000000 non-null float64
trip_total                1000000 non-null float64
dtypes: datetime64[ns](2), float64(5), int64(1), object(3)
memory usage: 83.9+ MB


In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
df.to_csv('rideshare.csv', index=False)